# SIF Downscaling v2 (including time dimenssion)

## Dependencies

In [4]:
using Pkg

Pkg.activate("/Net/Groups/BGI/work_3/OEMC/oemc_sif/bin")

using LinearAlgebra, Optim, Plots, Dates

using DimensionalData, YAXArrays, Zarr, Statistics, LineSearches

using Rasters: Center
using Rasters

using MLJ, DataFrames, Tidier, StatsPlots, PlotlyJS

  Activating project at `/Net/Groups/BGI/work_3/OEMC/oemc_sif/bin`


LoadError: UndefVarError: `ls` not defined

## Data

In [5]:
### SIF low resolution
sif_cube = Cube(open_dataset("/Net/Groups/BGI/work_3/OEMC/oemc_sif/data/SIF_TROPOMI_gridded_10km_filtered_europe_2018-05-04_2020-12-26.zarr/"))

### lst

lst_cube = Cube(open_dataset("/Net/Groups/BGI/work_3/OEMC/oemc_sif/data/LST_Sentinel3_gridded_1km_europe_2018-01-04_2020-12-29.zarr/"))


### (WATER) MODIS NDWI

modis_cube = Cube(open_dataset("/Net/Groups/BGI/work_3/OEMC/oemc_sif/data/MC43A4_gridded_1km_europe_2018-05-01_2020-12-31_YAXA.zarr/"))

### (VEGETATION) Sentinel-3 OTCI (Terrestrial Chlorophyll Index)


sentinel_3_cube = Cube(open_dataset("/Net/Groups/BGI/work_3/OEMC/oemc_sif/data/OLCI_Sentinel3_gridded_1km_europe_2018-01-04_2020-12-29.zarr/"))

4800×4800×138×10 YAXArray{Float32,4} with dimensions: 
  X Sampled{Int64} 1:1:4800 ForwardOrdered Regular Points,
  Y Sampled{Int64} 1:1:4800 ForwardOrdered Regular Points,
  Ti Sampled{DateTime} DateTime[2018-01-04T00:00:00, …, 2020-12-29T00:00:00] ForwardOrdered Irregular Points,
  Dim{:Variable} Categorical{String} String[RC865, lat, …, OGVI_sigma, RC681_sigma] Unordered
Total size: 118.45 GB


Computing NDWI

In [6]:
function ndwi(out, b4,b2)
    if !isnan.(b4) && !isnan.(b2) && !isinf.(b4) && !isinf.(b2)
        out .= (b4[:]-b2[:])/(b4[:] + b2[:])

    else
        out .= NaN32
    end
end

indims = (InDims(), InDims())

outdims = OutDims(Dim{:Variable}(["NDWI"]))

OutDims((YAXArrays.ByValue(Dim{:Variable, Vector{String}}(["NDWI"])),), :auto, Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}(), false, Array, :input, 1)

In [ ]:
ndwi_cube_high = mapCube(ndwi, (modis_cube[Variable = At("4")], modis_cube[Variable = At("2")]), indims = indims, outdims=outdims)

Progress:  26%|██████████▋                              |  ETA: 0:44:16

In [ ]:
# quick check
heatmap(ndwi_cube_high.data[:,:,3,1])